#Preprocessing

In [1]:
from zipfile import ZipFile as zf

with zf('data.zip','r') as zin:
    zin.extractall('./data/')

In [104]:
import os
os.chdir('./data/')

In [105]:
import pandas as pd
import py_midicsv as pm

paths = pd.read_csv('./maestro-v3.0.0.csv')

paths['midi_filename'] = paths['midi_filename'].apply(lambda s:s.split('/')[1])
paths['canonical_composer'] = paths['canonical_composer'].apply(lambda s:s.split()[-1])
songs = paths['midi_filename']
paths = paths.set_index(['midi_filename'])
paths

,canonical_composer,canonical_title,split,year,audio_filename,duration
midi_filename,,,,,,
MIDI-Unprocessed_Chamber3_MID--AUDIO_10_R3_2018_wav--1.midi,Berg,Sonata Op. 1,train,2018,2018/MIDI-Unprocessed_Chamber3_MID--AUDIO_10_R...,698.661160
MIDI-Unprocessed_03_R2_2008_01-03_ORIG_MID--AUDIO_03_R2_2008_wav--2.midi,Berg,Sonata Op. 1,train,2008,2008/MIDI-Unprocessed_03_R2_2008_01-03_ORIG_MI...,759.518471
MIDI-Unprocessed_066_PIANO066_MID--AUDIO-split_07-07-17_Piano-e_3-02_wav--3.midi,Berg,Sonata Op. 1,train,2017,2017/MIDI-Unprocessed_066_PIANO066_MID--AUDIO-...,464.649433
MIDI-Unprocessed_XP_21_R1_2004_01_ORIG_MID--AUDIO_21_R1_2004_01_Track01_wav.midi,Scriabin,"24 Preludes Op. 11, No. 13-24",train,2004,2004/MIDI-Unprocessed_XP_21_R1_2004_01_ORIG_MI...,872.640588
MIDI-Unprocessed_17_R1_2006_01-06_ORIG_MID--AUDIO_17_R1_2006_04_Track04_wav.midi,Scriabin,"3 Etudes, Op. 65",validation,2006,2006/MIDI-Unprocessed_17_R1_2006_01-06_ORIG_MI...,397.857508
...,...,...,...,...,...,...
MIDI-Unprocessed_XP_14_R1_2004_04_ORIG_MID--AUDIO_14_R1_2004_05_Track05_wav.midi,Mozart,"Sonata in F Major, K280",test,2004,2004/MIDI-Unprocessed_XP_14_R1_2004_04_ORIG_MI...,241.470442
MIDI-Unprocessed_XP_14_R1_2004_04_ORIG_MID--AUDIO_14_R1_2004_06_Track06_wav.midi,Mozart,"Sonata in F Major, K280",train,2004,2004/MIDI-Unprocessed_XP_14_R1_2004_04_ORIG_MI...,114.696243
MIDI-Unprocessed_SMF_12_01_2004_01-05_ORIG_MID--AUDIO_12_R1_2004_03_Track03_wav--1.midi,Mozart,"Sonata in F Major, K533",validation,2004,2004/MIDI-Unprocessed_SMF_12_01_2004_01-05_ORI...,1139.198478


In [106]:
# Fix piece collaborations (reduce composers)

paths.groupby(['canonical_composer']).count().nlargest(5,'split')

,canonical_title,split,year,audio_filename,duration
canonical_composer,,,,,
Chopin,201,201,201,201,201
Schubert,186,186,186,186,186
Liszt,150,150,150,150,150
Beethoven,146,146,146,146,146
Bach,145,145,145,145,145


In [30]:
os.mkdir('../csv/')
for song in songs:
    # Load the MIDI file and parse it into CSV format
    csv_data = pm.midi_to_csv(f'./{song}')
    with open(f'../csv/{song}.csv', "w") as f:
        f.writelines(csv_data)

#UnderstandingData

In [107]:
os.chdir('../')

In [108]:
from sklearn.decomposition import PCA

folder_path = './csv'
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

data = []

for file in csv_files:
    df = pd.read_csv(os.path.join(folder_path, file),error_bad_lines=False)
    df.columns = ['channel','time','action','glide','pitch','pedal']
    df = df.dropna()
    df.drop(['channel','glide','action'],axis=1,inplace=True)
    clf = PCA(n_components=0.9)
    new_ft = clf.fit_transform(df.transpose().to_numpy())
    df = pd.DataFrame(new_ft.T,columns=['pca1','pca2','pca3'])
    df['midi_filename'] = file.split('.csv')[0]
    df = df.set_index(['midi_filename'])
    data.append(df)

df = pd.concat(data)

C:\Users\W\AppData\Local\Temp\ipykernel_15624\2851879509.py:9: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv(os.path.join(folder_path, file),error_bad_lines=False)
Skipping line 4: expected 6 fields, saw 7

C:\Users\W\AppData\Local\Temp\ipykernel_15624\2851879509.py:9: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv(os.path.join(folder_path, file),error_bad_lines=False)
Skipping line 4: expected 6 fields, saw 7

C:\Users\W\AppData\Local\Temp\ipykernel_15624\2851879509.py:9: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv(os.path.join(folder_path, file),error_bad_lines=False)
Skipping line 4: expected 6 fields, saw 7

C:\Users\W\AppData\Local\Temp\ipykernel_156

In [110]:
df = pd.merge(df.reset_index(),paths.reset_index()).set_index('midi_filename')[['pca1','pca2','pca3','canonical_composer']]

In [113]:
# Only use top 5 composers
df = df[df['canonical_composer'].isin(['Chopin','Schubert','Beethoven','Bach','Liszt'])]

In [133]:
from sklearn.preprocessing import OneHotEncoder
import numpy as np

X = df.drop(['canonical_composer'],axis=1)
# y = df['canonical_composer']
y = OneHotEncoder().fit_transform(df['canonical_composer'].to_numpy().reshape(-1,1)).toarray()

#WithoutML

#NoFreeLunch

In [134]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=42,shuffle=True,test_size=0.2)
X_train,X_val,y_train,y_val = train_test_split(X_train,y_train,random_state=42,shuffle=True,test_size=0.2)

In [137]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

clf = KNeighborsClassifier()

gs = GridSearchCV(clf,{'n_neighbors':[x for x in np.arange(1,100)]},cv=5,scoring='accuracy')

gs.fit(X_train,y_train)
gs.best_estimator_

KNeighborsClassifier(n_neighbors=1)

In [139]:
gs.best_score_

0.285570530098832

In [140]:
y_pred_val = gs.predict(X_val)

list(zip(y_val,y_pred_val))

[(array([0., 1., 0., 0., 0.]), array([0., 0., 1., 0., 0.])),
 (array([0., 0., 1., 0., 0.]), array([0., 0., 1., 0., 0.])),
 (array([0., 1., 0., 0., 0.]), array([0., 1., 0., 0., 0.])),
 (array([0., 0., 0., 0., 1.]), array([1., 0., 0., 0., 0.])),
 (array([0., 0., 0., 1., 0.]), array([0., 1., 0., 0., 0.])),
 (array([0., 0., 0., 0., 1.]), array([0., 1., 0., 0., 0.])),
 (array([0., 0., 1., 0., 0.]), array([0., 0., 1., 0., 0.])),
 (array([0., 0., 0., 1., 0.]), array([0., 1., 0., 0., 0.])),
 (array([1., 0., 0., 0., 0.]), array([0., 1., 0., 0., 0.])),
 (array([0., 0., 1., 0., 0.]), array([0., 0., 1., 0., 0.])),
 (array([0., 0., 0., 0., 1.]), array([0., 1., 0., 0., 0.])),
 (array([1., 0., 0., 0., 0.]), array([0., 0., 1., 0., 0.])),
 (array([0., 0., 0., 1., 0.]), array([1., 0., 0., 0., 0.])),
 (array([1., 0., 0., 0., 0.]), array([1., 0., 0., 0., 0.])),
 (array([0., 1., 0., 0., 0.]), array([0., 0., 1., 0., 0.])),
 (array([0., 0., 1., 0., 0.]), array([0., 1., 0., 0., 0.])),
 (array([0., 1., 0., 0.,